In [1]:
from collections import defaultdict
import pandas as pd
import random
from collections import defaultdict
from datetime import datetime
from tqdm import tqdm

### 使用データセット

In [2]:
dataset = 'ml-1m'

In [3]:
data_path = '{}/ratings.dat'.format(dataset)

In [4]:
df = pd.read_table(data_path, sep="::", header=None,
                names=['user_id', 'item_id', 'rating', 'time_stamp'], index_col=False)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: read_table is deprecated, use read_csv instead.
  
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


In [5]:
# df = df[0:10000]

In [6]:
df.head(5)

,user_id,item_id,rating,time_stamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [7]:
users = df['user_id']
items = df['item_id']
times = df['time_stamp']

### item/user の条件設定

In [8]:
# item/userの固定フラグ
is_item_per_user_num_const = True
# 7,12,17, 52,102 で作成
item_per_user = 102

item/userを一定に整形

In [9]:
if is_item_per_user_num_const:
    print('is_item_per_user_num_const = {}'.format(is_item_per_user_num_const))
    print('item/user = {}'.format(item_per_user))
    
    User_all = defaultdict(list)
    User = defaultdict(list)
    
    for u, i, t in zip(users, items, times):
        User_all[u].append([i,t])
        
    # 時系列順にソートする
    for userid in User.keys():
        User[userid].sort(key=lambda x: x[1])
        
    for u in User_all:
        items = User_all[u]
        if len(items) > item_per_user:
            items = items[0:item_per_user]
            User[u] = items
    
    users = []
    items = []
    times = []
    
    for u in User:
        for values in User[u]:
            users.append(u)
            items.append(values[0])
            times.append(values[1])

is_item_per_user_num_const = True
item/user = 102


In [10]:
print('item/user > {} を満たすユーザ数:{}'.format(item_per_user, len(set(users))))

item/user > 102 を満たすユーザ数:2857


### user数 の条件設定

In [11]:
# ユーザ数の指定フラグ
is_user_num_const = False
user_num = 6000

指定したユーザ数にデータを整形

In [12]:
if is_user_num_const:
    print('is_user_num_const = {}'.format(is_user_num_const))
    
    unique_users = df['user_id'].unique() 
    master_user = random.sample(list(unique_users), user_num)
    ignore_user = []
    
    users_ = users
    items_ = items
    times_ = times
    
    users = []
    items = []
    times = []
    
    for u, i, t in tqdm(zip(users_, items_, times_)):
        if u in master_user:
            users.append(u)
            items.append(i)
            times.append(t)

In [13]:
# 評価件数カウント用のdict
countU = defaultdict(lambda: 0)

In [14]:
for user, item, time in zip(users, items, times):
    countU[user] += 1

In [15]:
usermap = dict()
usernum = 0
itemmap = dict()
itemnum = 0
User = dict()

In [16]:
for user, item, time in zip(users, items, times):
    #  評価件数がitem_per_user以下のものは無視    
    if countU[user] < item_per_user:
        continue
        
    if user in usermap:
        userid = usermap[user]
    else:
        usernum +=1
        userid = usernum
        usermap[user] = userid
        User[userid] = []
        
    if item in itemmap:
        itemid = itemmap[item]
    else:
        itemnum += 1
        itemid = itemnum
        itemmap[item] = itemid
        
    User[userid].append([time, itemid])

In [17]:
# 時系列順にソートする
for userid in User.keys():
    User[userid].sort(key=lambda x: x[0])

In [18]:
print (usernum, itemnum)

2857 3330


In [19]:
usermap
user_ids = list(usermap.keys())
fixed_user_ids = list(usermap.values())

In [20]:
df_user_ids = pd.DataFrame(
    data={
        'user_ids' : user_ids,
        'fixed_user_ids' : fixed_user_ids
    },
    columns=['user_ids', 'fixed_user_ids']
)

In [21]:
itemmap
item_ids = list(itemmap.keys())
fixed_item_ids = list(itemmap.values())

In [22]:
df_item_ids = pd.DataFrame(
    data={
        'item_ids' : item_ids,
        'fixed_item_ids' : fixed_item_ids
    },
    columns=['item_ids', 'fixed_item_ids']
)

In [23]:
user_list = []
item_list = []

for user in User:
    items = User[user]
    for item in items:
        user_list.append(user)
        item_list.append(item[1])

In [24]:
each_item_num = '_ItemPerUser=' + str(item_per_user) if is_item_per_user_num_const else ''
user_num_in_data= '_UserNum=' + str(user_num) if is_user_num_const else ''

fiexd_df =  pd.DataFrame(
            data={
                'user_id': user_list,
                'item_id': item_list
            }
)

In [25]:
df_user_ids.to_csv("{}/processing_data/{}_usermap{}{}.csv".format(dataset, dataset,user_num_in_data, each_item_num))
df_item_ids.to_csv("{}/processing_data/{}_itemmap{}{}.csv".format(dataset, dataset,user_num_in_data, each_item_num))
fiexd_df.to_csv("{}/processing_data/{}_processing{}{}.txt".format(dataset, dataset,user_num_in_data, each_item_num), header=None, index=False, sep=' ')